<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [25]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math

In [3]:
## Load the model
model = tf.keras.models.load_model('../models/model_v2.keras')

# Show the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 20, 6)          │            13 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 256)        │       269,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 20, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,976,021 (11.35 MB)

 Trainable params: 992,002 (3.78 MB)

 Non-trainable params: 13 (56.00 B)

 Optimizer params: 1,984,006 (7.57 MB)

In [4]:
## Load numpy test data
x_test = np.load('../x_test.npy')
y_test = np.load('../y_test.npy')
mean_variance = np.load('../mean_variance_v2.npy')

In [5]:
## IMPORTANT: Set up denormalizer
mean = mean_variance[0]
variance = mean_variance[1]

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [44]:
## ONE STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 1

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 6)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
past_lat = []
past_lon = []

# Get route coordinates
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = pred_coords.numpy()[0][0]
pred_lon = pred_coords.numpy()[0][1]

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]


# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[pred_lat],
  lon=[pred_lon],
  name="Predicted Point",
  marker={'size': 15, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Prediction:
tf.Tensor([[ -6.2823973 106.872986 ]], shape=(1, 2), dtype=float32)
True Coordinate:
tf.Tensor([[ -6.2821584 106.87589  ]], shape=(1, 2), dtype=float32)


In [13]:
def calculate_speed(lat1, lon1, lat2, lon2):
  """
  Calculates the speed (in m/s) of an object moving between two WGS84 coordinates.

  Args:
      lat1: Latitude of the first coordinate (in degrees).
      lon1: Longitude of the first coordinate (in degrees).
      lat2: Latitude of the second coordinate (in degrees).
      lon2: Longitude of the second coordinate (in degrees).

  Returns:
      The speed of the object in meters per second.
  """
  # Convert coordinates to radians
  lat1 = math.radians(lat1)
  lon1 = math.radians(lon1)
  lat2 = math.radians(lat2)
  lon2 = math.radians(lon2)

  # Earth radius (in meters)
  earth_radius = 6371e3

  # Calculate the central angle between the points
  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) * math.sin(dlon / 2)
  c = 2 * math.asin(math.sqrt(a))

  # Distance between the points (in meters)
  distance = earth_radius * c

  # Time (in seconds)
  time = 60  # 1 minute

  # Speed (in m/s)
  speed = distance / time

  return speed

In [86]:
def calculate_bearing(lat1, lon1, lat2, lon2):
  """
  Calculates the initial bearing of an object moving between two WGS84 coordinates.

  Args:
      lat1: Latitude of the first coordinate (in degrees).
      lon1: Longitude of the first coordinate (in degrees).
      lat2: Latitude of the second coordinate (in degrees).
      lon2: Longitude of the second coordinate (in degrees).

  Returns:
      The initial bearing of the object in degrees (0: north, 90: east, 180: south, 270: west).
  """
  # Convert coordinates to radians
  lat1 = math.radians(lat1)
  lon1 = math.radians(lon1)
  lat2 = math.radians(lat2)
  lon2 = math.radians(lon2)

  y = math.sin(lon2 - lon1) * math.cos(lat2)
  x1 = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(lon2 - lon1)
  x2 = math.atan2(y, x1)

  # Convert angle from radians to degrees
  bearing = (math.degrees(x2) + 360) % 360

  return bearing

In [93]:
## MULTI STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 16
steps = 60

# Predict the next "step" coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
x_data = x_test[num] # Data that will be fed to the model

pred_coords = []
for i in range(steps):
  current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 6))).numpy().tolist()[0]
  pred_coords.append(current_pred)

  x_data = np.delete(x_data, 0, 0)

  speed = calculate_speed(x_data[-1][0], x_data[-1][1], x_data[-2][0], x_data[-2][1])
  bearing = calculate_bearing(x_data[-1][0], x_data[-1][1], x_data[-2][0], x_data[-2][1])
  hour = x_data[-1][4]
  day = x_data[-1][5]

  current_pred.extend((speed, bearing, hour, day))
  
  x_data = np.append(x_data, [current_pred], axis=0)

true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


### Map display of True Point vs Predicted Point

# Get route coordinates
past_lat = []
past_lon = []
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []
for point in pred_coords:
  pred_lat.append(point[0])
  pred_lon.append(point[1])

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]


# Build the map
fig = go.Figure()

# Plot previous path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

# Plot predicted path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

In [78]:
x_data[4]

array([ -6.14196129, 106.89106266,  19.73520815, 192.47916667,
         7.        ,   3.        ])

In [72]:
x_data = x_test[num]

current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 6))).numpy().tolist()[0]
speed = calculate_speed(x_data[-1][0], x_data[-1][1], x_data[-2][0], x_data[-2][1])
bearing = calculate_bearing(x_data[-1][0], x_data[-1][1], x_data[-2][0], x_data[-2][1])
current_pred.append(speed)
current_pred.append(bearing)
print(current_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[-6.282397270202637, 106.87298583984375, 21.032599431849786, 359.9871681399021]
